In [1]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = PBSCluster(cores=4, memory="20GB", project='PangeoKEspectrumeNATL60', walltime='04:00:00')
cluster.scale(12)
cluster

/work/logiciels/rh7/conda/4.5.4/envs/pangeo_full_201902/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://10.120.43.58:39916 Dashboard: http://10.120.43.58:52952/status,Cluster Workers: 12 Cores: 48 Memory: 240.00 GB


In [4]:
import sys, glob
import numpy as np
import xarray as xr
import xscale.spectral.fft as xfft
import xscale 
import Wavenum_freq_spec_func as wfs
import time

In [5]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
from matplotlib.colors import LogNorm

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic


In [7]:
%%time
dsuJ=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridU-2D/eNATL60-BLBT02_1h_*200907*_gridU-2D.nc',parallel=True)
dsvJ=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridV-2D/eNATL60-BLBT02_1h_*200907*_gridV-2D.nc',parallel=True)


CPU times: user 48.1 s, sys: 14.1 s, total: 1min 2s
Wall time: 3min 21s


In [8]:
dsuJ

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 744, x: 8354, y: 4729)
Coordinates:
    nav_lat               (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    nav_lon               (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    time_centered         (time_counter) datetime64[ns] dask.array<shape=(744,), chunksize=(24,)>
  * time_counter          (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-07-31T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    time_centered_bounds  (time_counter, axis_nbounds) float64 dask.array<shape=(744, 2), chunksize=(24, 2)>
    time_counter_bounds   (time_counter, axis_nbounds) float64 dask.array<shape=(744, 2), chunksize=(24, 2)>
    sozocrtx              (time_counter, y, x) float32 dask.array<shape=(744, 4729, 8354), chunksize=(24, 4729, 8354)>
    bozocrtx              (time_counter, y, x) float32 dask.array<shape=(744, 4729, 8354), chunksize=(24, 4729, 8354)>
Attrib

In [9]:
%%time
lat=dsuJ['nav_lat']
lon=dsuJ['nav_lon']
 
latmin = 40.0; latmax = 45.0;
lonmin = -40.0; lonmax = -35.0;

domain = (lonmin<lon) * (lon<lonmax) * (latmin<lat) * (lat<latmax)
where = np.where(domain)

#get indice
jmin = np.min(where[0][:])
jmax = np.max(where[0][:])
imin = np.min(where[1][:])
imax = np.max(where[1][:])

latbox=lat[jmin:jmax,imin:imax]
lonbox=lon[jmin:jmax,imin:imax]


CPU times: user 673 ms, sys: 204 ms, total: 878 ms
Wall time: 808 ms


In [10]:
%%time
print('Open netcdf for J')
dsuJ=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridU-2D/eNATL60-BLBT02_1h_*200907*_gridU-2D.nc',parallel=True)
dsvJ=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridV-2D/eNATL60-BLBT02_1h_*200907*_gridV-2D.nc',parallel=True)

print('Select dates for J')
u_J=dsuJ.sel(time_counter=slice('2009-07-01','2009-07-31'))['sozocrtx']
v_J=dsvJ.sel(time_counter=slice('2009-07-01','2009-07-31'))['somecrty']


print('Select box area for J')
u_J_box=u_J[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})
v_J_box=v_J[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})

print('Open netcdf for A')
dsuA=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridU-2D/eNATL60-BLBT02_1h_*200908*_gridU-2D.nc',parallel=True)
dsvA=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridV-2D/eNATL60-BLBT02_1h_*200908*_gridV-2D.nc',parallel=True)

print('Select dates for A')
u_A=dsuA.sel(time_counter=slice('2009-08-01','2009-08-31'))['sozocrtx']
v_A=dsvA.sel(time_counter=slice('2009-08-01','2009-08-31'))['somecrty']


print('Select box area for A')
u_A_box=u_A[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})
v_A_box=v_A[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})

print('Open netcdf for S')
dsuS=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridU-2D/eNATL60-BLBT02_1h_*200909*_gridU-2D.nc',parallel=True)
dsvS=xr.open_mfdataset('/work/ALT/odatis/eNATL60/BLBT02/gridV-2D/eNATL60-BLBT02_1h_*200909*_gridV-2D.nc',parallel=True)

print('Select dates for S')
u_S=dsuS.sel(time_counter=slice('2009-09-01','2009-09-30'))['sozocrtx']
v_S=dsvS.sel(time_counter=slice('2009-09-01','2009-09-30'))['somecrty']


print('Select box area for S')
u_S_box=u_S[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})
v_S_box=v_S[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})

print('Concat data array')
u_JAS_box=xr.concat([u_J_box,u_A_box,u_S_box],dim='time_counter')
v_JAS_box=xr.concat([v_J_box,v_A_box,u_S_box],dim='time_counter')

# - get dx and dy
print('get dx and dy')
dx_JAS,dy_JAS = wfs.get_dx_dy(u_JAS_box[0],lonbox,latbox)


#... Detrend data in all dimension ...
print('Detrend data in all dimension')
u_JAS = wfs.detrendn(u_JAS_box,axes=[0,1,2])
v_JAS = wfs.detrendn(v_JAS_box,axes=[0,1,2])

#... Apply hanning windowing ...') 
print('Apply hanning windowing')
u_JAS = wfs.apply_window(u_JAS, u_JAS.dims, window_type='hanning')
v_JAS = wfs.apply_window(v_JAS, v_JAS.dims, window_type='hanning')


# - get derivatives
print('velocity derivatives')
derivatives_JAS = wfs.velocity_derivatives(u_JAS, v_JAS, xdim='x', ydim='y', dx={'x': dx_JAS, 'y': dy_JAS})
dudx_JAS = derivatives_JAS['u_x']; dudy_JAS = derivatives_JAS['u_y']
dvdx_JAS = derivatives_JAS['v_x']; dvdy_JAS = derivatives_JAS['v_y']

# - compute terms
print('computer terms')
phi1_JAS = u_JAS*dudx_JAS + v_JAS*dudy_JAS
phi2_JAS = u_JAS*dvdx_JAS + v_JAS*dvdy_JAS

print('fft u v ')
u_JAShat = xfft.fft(u_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)
v_JAShat = xfft.fft(v_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)

print('fft phi')
phi1_JAS_hat = xfft.fft(phi1_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)
phi2_JAS_hat = xfft.fft(phi2_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)

print('multiply')
tm1_JAS = (u_JAShat.conj())*phi1_JAS_hat
tm2_JAS = (v_JAShat.conj())*phi2_JAS_hat

# - computer transfer
print('Compute transfer')
Nk_JAS,Nj_JAS,Ni_JAS = u_JAS.shape
transfer_2D_JAS = -1.0*(tm1_JAS + tm2_JAS)/np.square(Ni_JAS*Nj_JAS)
transfer_term_JAS = transfer_2D_JAS.real

#... Get frequency and wavenumber ... 
print('Get frequency and wavenumber')
ffrequency_JAS = u_JAShat.f_time_counter
kx_JAS = u_JAShat.f_x
ky_JAS = u_JAShat.f_y

#... Get istropic wavenumber ... 
print('Get istropic wavenumber')
wavenumber_JAS,kradial_JAS = wfs.get_wavnum_kradial(kx_JAS,ky_JAS)

#... Get numpy array ... 
print('Get numpy array')
var_psd_np_JAS = transfer_term_JAS.values

#... Get 2D frequency-wavenumber field ... 
print('Get transfer')
transfer_JAS = wfs.get_f_k_in_2D(kradial_JAS,wavenumber_JAS,var_psd_np_JAS) 

print('Get flux')
flux_JAS = wfs.get_flux_in_1D(kradial_JAS,wavenumber_JAS,var_psd_np_JAS)


Open netcdf for J
Select dates for J
Select box area for J
Open netcdf for A
Select dates for A
Select box area for A
Open netcdf for S
Select dates for S
Select box area for S
Concat data array
get dx and dy
Detrend data in all dimension
Apply hanning windowing
velocity derivatives
computer terms
fft u v 
fft phi
multiply
Compute transfer
Get frequency and wavenumber
Get istropic wavenumber
Get numpy array
Get transfer
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
Get flux
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
CPU times: user 20min 28s, sys: 2min, total: 22min 28s
Wall time: 30min 3s


In [11]:
# Save to Netscdf file
# - build dataarray
print('Save to Netscdf file')
transfer_JAS_da = xr.DataArray(transfer_JAS,dims=['frequency','wavenumber'],name="transfer",coords=[ffrequency_JAS ,wavenumber_JAS])
flux_JAS_da = xr.DataArray(flux_JAS,dims=['frequency','wavenumber'],name="flux",coords=[ffrequency_JAS,wavenumber_JAS])
transfer_JAS_da.attrs['Name'] = 'KE_Transfer_Flux_JAS_w_k_from_1h_eNATL60-BLBT02.nc'

transfer_JAS_da.to_dataset().to_netcdf(path='KE_Transfer_Flux_JAS_w_k_from_1h_eNATL60-BLBT02.nc',mode='w',engine='scipy')
flux_JAS_da.to_dataset().to_netcdf(path='KE_Transfer_Flux_JAS_w_k_from_1h_eNATL60-BLBT02.nc',mode='a',engine='scipy')


Save to Netscdf file
